In [9]:
import IPython.display as ipd
# import matplotlib.pyplot as plt
import numpy as np
import scipy.signal

import simulator

import os
import soundfile as sf
from scipy.signal import resample_poly

In [10]:
def create_brir(src_azim, src_elev):
    room_materials = [1, 1, 1, 1, 15, 16]
    room_dim_xyz = [5, 4, 3]
    head_pos_xyz = [2, 1.4, 1.2]
    head_azim = 0
    src_elev = src_elev
    src_azim = src_azim
    src_dist = 1.4

    c = 344.5
    buffer = 0
    sr = 44100
    dur = 0.5
    use_hrtf_symmetry = True
    use_highpass = True
    use_jitter = False
    use_log_distance = False
    hrtf_locs = None
    hrtf_firs = None
    incorporate_lead_zeros = False

    brir = simulator.get_brir(
        room_materials=room_materials,
        room_dim_xyz=room_dim_xyz,
        head_pos_xyz=head_pos_xyz,
        head_azim=head_azim,
        src_azim=src_azim,
        src_elev=src_elev,
        src_dist=src_dist,
        buffer=buffer,
        sr=sr,
        dur=dur,
        use_jitter=use_jitter,
        use_hrtf_symmetry=use_hrtf_symmetry,
        incorporate_lead_zeros=incorporate_lead_zeros,
        strict=True,
    )

    return brir

In [11]:
def spatialise(y, brir):
    y_padded = np.pad(y, (brir.shape[0] - 1, 0))
    y_spatialized = scipy.signal.oaconvolve(
        y_padded[:, None],
        brir,
        mode="valid",
        axes=0,
    )

    return y_spatialized

In [12]:
# azim = np.linspace(180,-180,72, endpoint=False)
azim = np.linspace(-180, 180, 72, endpoint=False)
azim

array([-180., -175., -170., -165., -160., -155., -150., -145., -140.,
       -135., -130., -125., -120., -115., -110., -105., -100.,  -95.,
        -90.,  -85.,  -80.,  -75.,  -70.,  -65.,  -60.,  -55.,  -50.,
        -45.,  -40.,  -35.,  -30.,  -25.,  -20.,  -15.,  -10.,   -5.,
          0.,    5.,   10.,   15.,   20.,   25.,   30.,   35.,   40.,
         45.,   50.,   55.,   60.,   65.,   70.,   75.,   80.,   85.,
         90.,   95.,  100.,  105.,  110.,  115.,  120.,  125.,  130.,
        135.,  140.,  145.,  150.,  155.,  160.,  165.,  170.,  175.])

In [13]:
elev = np.linspace(0,60,7, endpoint=True)
elev

array([ 0., 10., 20., 30., 40., 50., 60.])

In [ ]:
# Configuration
sound_path = '/mnt/lustre/work/macke/mwe234/datasets/short_audio_500'
save_folder = '/mnt/lustre/work/macke/mwe234/datasets/simulated/500_spatial_360_7_test/'
os.makedirs(save_folder, exist_ok=True)

audio_files = [f for f in os.listdir(sound_path) if f.endswith('.wav')]

# spatialise
azim = np.linspace(-180, 180, 72, endpoint=False)
elev = np.linspace(0,60,7, endpoint=True)
# elev = [0]

for a in azim:
    for e in elev:
        print(f'azim: {a}, elev: {e}')
        brir = create_brir(a, e)

        for file in audio_files:
            print(file)
            sound = os.path.join(sound_path, file)
            y, sr = sf.read(sound)

            # use mono channel if stereo
            if len(y.shape) > 1:
                y_mono = y[:,1]
            else:
                y_mono = y

            # up-sample
            y_resampled = resample_poly(y_mono, up=48000, down=44100)
            y = y_resampled[:2*48000]

            y_spatialized = spatialise(y, brir)
            y_spatialized = np.asarray(y_spatialized, dtype=np.float32)
            filename = file.split('.')[0]
            
            # os.makedirs(save_folder, exist_ok=True)
            np.save(save_folder + f"sound_{filename}_azim{round(a)}_elev{round(e)}.npy", y_spatialized)
            # print(f"sound_{filename}_a{a}_e{e}.npy")
        
        # print(f'sound_{i}_a{a}_e{e}')
        ipd.display(ipd.Audio(rate=sr, data=y_spatialized.T))

azim: -180.0, elev: 0.0
[get_brir] loaded kemar_hrtfs (Gardner & Martin, 1994): (368, 128, 2)
[get_brir] head_pos: [2.0, 1.4, 1.2], src_pos: [0.6000000000000001, 1.3999999999999997, 1.2], room_dim: [5, 4, 3]
[get_brir] time elapsed: 22.13864302635193 seconds
208030-3-4-0.wav
74810-9-1-12.wav
77774-4-0-1.wav
189982-0-0-4.wav
119455-5-0-7.wav
203929-7-5-1.wav
197554-2-0-82.wav
46391-1-0-0.wav
106015-5-0-0.wav
98263-9-0-22.wav
196561-3-0-16.wav
83502-0-0-11.wav
49312-2-0-10.wav
28284-3-0-0.wav
71087-4-0-0.wav
52882-2-0-4.wav
185375-9-0-61.wav
174994-3-0-0.wav
165643-4-0-0.wav
75743-0-0-6.wav
129750-2-0-36.wav
159748-8-0-5.wav
165644-4-0-0.wav
40722-8-0-1.wav
115411-3-2-0.wav
175904-2-0-12.wav
178261-7-3-4.wav
42937-4-0-0.wav
55013-3-0-7.wav
124389-8-1-2.wav
66324-9-0-4.wav
164311-3-1-0.wav
24728-7-2-0.wav
71086-4-0-0.wav
207214-2-0-54.wav
72724-3-2-8.wav
176787-5-0-9.wav
122738-9-0-11.wav
105029-7-1-11.wav
165192-9-0-5.wav
184575-3-0-0.wav
144351-4-3-15.wav
143651-2-0-0.wav
74495-3-0-1.wa

azim: -180.0, elev: 10.0
[get_brir] loaded kemar_hrtfs (Gardner & Martin, 1994): (368, 128, 2)
[get_brir] head_pos: [2.0, 1.4, 1.2], src_pos: [0.6212691457829089, 1.3999999999999997, 1.4431074487337023], room_dim: [5, 4, 3]


Process ForkPoolWorker-9:
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
